In [12]:
import sys
import pandas as pd

import importlib
import src.enrichment.generate_negatives
importlib.reload(src.enrichment.generate_negatives)
from src.enrichment.generate_negatives import generate_negative_cases



from pathlib import Path
project_root = Path().resolve().parent.parent
sys.path.append(str(project_root))

from src.io import save_processed_data

# Load enriched incident data (positives)
df_pos = pd.read_parquet("../../data/processed/incidents_enriched.parquet")
df_pos["incident_occurred"] = True

# Drop simulated feature with spurious correlation (if present)
df_pos = df_pos.drop(columns=[col for col in df_pos.columns if col == "simulated_medical_condition"])

# Remove 'simulated_medical_condition' from generate_negative_cases logic if needed
# This call assumes the generate_negative_cases function does not require that column
n_neg = len(df_pos) * 50
df_neg = generate_negative_cases(df_pos, n_negatives=n_neg)

# Also drop the column in negatives if it was injected
if "simulated_medical_condition" in df_neg.columns:
    df_neg = df_neg.drop(columns=["simulated_medical_condition"])

df_neg["incident_occurred"] = False

# Combine and save
df_all = pd.concat([df_pos, df_neg], ignore_index=True)
save_processed_data(df_all, "../../data/processed/model_ready_balanced.parquet")

print("✅ Dataset balanced and saved with 1:50 ratio (without medical_condition)")
print("Positives:", len(df_pos))
print("Negatives:", len(df_neg))
print("Total:", len(df_all))


✅ Dataset balanced and saved with 1:50 ratio (without medical_condition)
Positives: 682
Negatives: 34100
Total: 34782
